# Homework Assignment - Part B

## Imports

In [0]:
import random
import itertools
import math
import csv
import numpy as np
import scipy
from scipy import stats
from scipy.stats import norm, ttest_ind
import pickle

### Google Drive and the Yandex clicklog file

In [0]:
# mount google drive
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
# copy the yandex click log from google drive
!cp "/gdrive/My Drive/YandexRelPredChallenge.txt" .

## Experiment parameters

In [0]:
scale = 2  # the relevance scale (i.e. 2 = {0, 1} = binary, 3 = {0, 1, 2} = ternary, etc)
cutoff = 3  # the cutoff rank
theta = 0.15  # probability of not clicking on a relevant document
thresholds = [(0.05, 0.1), 
              (0.1, 0.2), 
              (0.2, 0.3), 
              (0.3, 0.4), 
              (0.4, 0.5), 
              (0.5, 0.6), 
              (0.6, 0.7), 
              (0.7, 0.8), 
              (0.8, 0.9), 
              (0.9, 0.9500001)]  # the grouping thresholds
p_overlapping_rel = 0.2  # the probability that a given pair of documents with the same relevance score will be overlapping
p_overlapping_not_rel = 0.2
overlapping_samples = 4  # the number of times we generate an ID for one pair (i.e. the number of potential overlapping combinations)

## Run the experiment

### Step 1: Simulate Rankings of Relevance for *E* and *P*


In [0]:
def simulate_pairs(scale, cutoff):
    """Generate pairs of rankings.

    Args:
        scale: the relevance scale (i.e. 2 = {0, 1} = binary, 3 = {0, 1, 2} = ternary, etc).
        cutoff: the cutoff rank.
    Returns:
        list of pairs of rankings.
    """
    rankings = np.array(list(itertools.product(range(scale), repeat=cutoff)))
    pairs = np.array(list(itertools.product(rankings, repeat=2)))
    return pairs

In [0]:
pairs = simulate_pairs(scale, cutoff)
print(f'Number of unique pairs: {len(pairs)}')

Number of unique pairs: 64


### Step 2 - Calculate the *delta measure* and group the results

#### Step 2a: Calculate the *delta measure*

In [0]:
def err(ranking, theta):
    """Function that calculates ERR according to "Expected Reciprocal Rank for Graded Relevance" (Chapelle et al.)
  
    Args:
        ranking: given ranking of documents with relevance scores
        theta: parameter of ERR calculation. Function of relevance
    Returns:
        err: expected reciprocal rank for given ranking
    """
    err = 0.0
    p = 1.0
    for i in range(len(ranking)):
        r = np.abs(ranking[i] - theta)
        err += p * r / (i + 1)
        p *= 1 - r
    return err


def calculate_measures(pairs, theta):
    """Calculate the measure (ERR) for all ranking pairs.

    Args:
        pairs: the ranking pairs
        theta: probability of not clicking on a relevant document
    Returns:
        list of tuples of (ranking pair, delta_err).
    """
    pairs_measures = []
    for pair in pairs:
        err_e = err(pair[0], theta)
        err_p = err(pair[1], theta)
        if err_e > err_p:
          delta_err = err_e - err_p
          pairs_measures.append(np.array([pair, delta_err]))

    return np.array(pairs_measures)

In [0]:
pairs_measures = calculate_measures(pairs, theta)
print(f'Number of unique pairs for which E outperforms P (in terms of the measure): {len(pairs_measures)}')
#print(f'Example (pair, measure) tuple: {list(random.choice(pairs_measures))}')

Number of unique pairs for which E outperforms P (in terms of the measure): 28


#### Step 2b: Group the results

In [0]:
def group_pairs_measures(pairs_measures, thresholds):
    """Group the ranking pairs on their measures (ERR).

    Args:
        pairs_measures: the (ranking pair, measure) tuples
        thresholds: the predefined group thresholds
    Returns:
        list of groups containing their ranking pairs.
    """
    grouping_dict = dict()
    for i in range(len(pairs_measures)):
        group = [j for j in range(len(thresholds)) if (pairs_measures[i][1] >= thresholds[j][0] and pairs_measures[i][1] < thresholds[j][1])]
        if len(group) > 0:
          grouping_dict[str(pairs_measures[i][0])] = group[0]
        else:
          grouping_dict[str(pairs_measures[i][0])] = None
    return grouping_dict

In [0]:
grouping_dict = group_pairs_measures(pairs_measures, thresholds)
print('Groups:')
for i in range(len(thresholds)):
    print(f'  {i+1: <2} ({thresholds[i][0]:.2f}, {thresholds[i][1]:.2f}): {list(grouping_dict.values()).count(i): <2} pairs')

Groups:
  1  (0.05, 0.10): 2  pairs
  2  (0.10, 0.20): 2  pairs
  3  (0.20, 0.30): 2  pairs
  4  (0.30, 0.40): 7  pairs
  5  (0.40, 0.50): 4  pairs
  6  (0.50, 0.60): 1  pairs
  7  (0.60, 0.70): 4  pairs
  8  (0.70, 0.80): 0  pairs
  9  (0.80, 0.90): 0  pairs
  10 (0.90, 0.95): 0  pairs


###  Step 3: Implement Team-Draft Interleaving and Probabilistic Interleaving

In [0]:
def gen_ids(pair, probability_of_overlapping_rel, probability_of_overlapping_not_rel):
    """ Function that generates IDs for documents in a pair of rankings.
    
    Args:
        pair: pair of rankings (numpy array of dimensions (2,3)).
        probability_of_overlapping_rel: probability that a pair of relevant documents will be overlapping (0<x<1)
        probability_of_overlapping_not_rel: probability that a pair of not relevant documents with the same relevance will be
        overlapping (0<x<1)
    Returns:
        ids: identifiers for pair of rankings (numpy array of dimensions (2,3), 0 for not overlapping, a>0 if overlapping and 
        relevant, a<0 if overlapping and not relevant)
    Additional notes:
        - we set probability_of_overlapping_rel = probability_of_overlapping_not_rel for simplification
  """
    ids = np.zeros((2,3))
    order_rel_e = np.random.permutation(np.where(pair[0,:] == 1)[0])
    order_rel_p = np.random.permutation(np.where(pair[1,:] == 1)[0])
    order_not_rel_e = np.random.permutation(np.where(pair[0,:] == 0)[0])
    order_not_rel_p = np.random.permutation(np.where(pair[1,:] == 0)[0])    
    overlappings_rel = np.min([np.min([sum(pair[0,:]), sum(pair[1,:])]), np.random.binomial(sum(pair[0,:]) * sum(pair[1,:]), probability_of_overlapping_rel)])
    overlappings_not_rel = np.min([np.min([sum(pair[0,:]), sum(pair[1,:])]), np.random.binomial((3 - sum(pair[0,:])) * (3 - sum(pair[1,:])), probability_of_overlapping_not_rel)])
    for i in range(overlappings_rel):
        ids[0, order_rel_e[i]] = i + 1
        ids[1, order_rel_p[i]] = (i + 1)     
    for j in range(overlappings_not_rel):
        ids[0, order_not_rel_e[j]] = - j - 1
        ids[1, order_not_rel_p[j]] = (-j - 1)
    total_overlappings = overlappings_not_rel + overlappings_rel
    return ids, total_overlappings
  
  
def softmax(vector):
    """Function calculates inverse softmax probabilities for a given vector.
    
    Args:
        vector: vector of lenght n
    Returns:
        probabilities: softmax probabilities.
    """
    probabilities = np.zeros((len(vector)))
    sum_ = np.sum(np.exp(-vector))
    for i in range(len(vector)):
        probabilities[i] = np.exp(-vector[i]) / sum_
    return probabilities
  
  
def algorithm_orders():
    """ Function that generates all possible orders of search engines for the interleaving process. 
    
    Args:
        none
    Returns:
        orders: all unique permutations of set (0,0,0,1,1,1)
    """
    orders = list(itertools.permutations([0,0,0,1,1,1]))
    
    return list(set(orders))
  
def probabilistic_document_order(pair, ids):
    """ Function that generates all possible orders in which a search engine will send its documents from given ranking, as well as 
        the probability of the given order being generated for probabilistic interleaving. Softmax over ranking is assumed.
    
    Args:
        pair: pair of rankings produced by production and experimental algorithms
        ids: identifiers determining if two documents are overlapping
    Returns:
        p_pair: all possible probabilistic orders produced by two algorithms
        p_ids: identifiers that share indices with p_pair, used to generalize the interleaving function
        probability_of_ranking_: probability of the pair of probabilistic orders being produced
    
    """
    p_pair = []
    p_ids = []
    p_prob = []
    prob_order = list(itertools.permutations(range(3)))
    prob_pairs = list(itertools.product(prob_order, repeat=2))
    probability_of_ranking = []
    for i in range(len(prob_order)):
        prob__ = prob_order[i]
        ranking_prob = 1
        for j in range(len(prob_order[0])):
            probs_ = softmax(np.asarray(prob__))
            ranking_prob *= probs_[0]
            prob__ = np.delete(prob__, 0)
        probability_of_ranking.append(ranking_prob)
    probability_of_ranking_ = np.asarray(list(itertools.product(probability_of_ranking, repeat=2)))
    probability_of_ranking_ = probability_of_ranking_[:,0] * probability_of_ranking_[:,1]
    for i in range(len(prob_pairs)):
        pair_i = np.zeros((2,3))
        ids_i = np.zeros((2,3))
        prob_i = np.zeros((2))
        for j in range(len(prob_pairs[0])):
            prob_i[j] = 1
            for k in range(len(prob_pairs[0][0])):
                pair_i[j,k] = pair[j, prob_pairs[i][j][k]]
                ids_i[j,k] = ids[j, prob_pairs[i][j][k]]
        p_pair.append(pair_i)
        p_ids.append(ids_i)
        
    return p_pair, p_ids, probability_of_ranking_
  
  
def interleaving(pair, ids, order, total_overlappings):
    """ Function performs the interleaving, given the:
            - order of search engines (unique permutations of [0,0,0,1,1,1])
            - order in which given search engine is sending its documents (probabilistic interleaving)
            - total overlappings in the ranking
        It is used for both team draft and probabilistic interleavings. 
            
    Args:
        pair: pair of rankings produced by two search engines. For probabilistic interleaving we interpret this input
        as order in which the documents from given ranking are sent (numpy array of dimensions (2,3))
        ids: identifiers of overlapping documents
        order: order of search engines (unique permutations of [0,0,0,1,1,1])
        total_overlappings: total count of overlapping cases, used to determine size of the interleaved list
    Returns: 
        interleaved: a matrix of size (number of unique documents x 3). First column shows the 
        relevance of document on the list (0 - not relevant; 1 - relevant), second column shows which algorithm 
        sent the document (0 - experimental; 1 - production), third column shows if the document was overlapping
        (0 - not overlapping; 1 - overlapping)
    
    """
    interleaved = np.zeros((6 - total_overlappings, 3))
    queue_e = pair[0,:]
    queue_p = pair[1,:]
    ids_e = ids[0,:]
    ids_p = ids[1,:]
    for i in range(6 - total_overlappings):
        if order[i] == 0 and len(queue_e) > 0:
            interleaved[i, 0] = queue_e[0]
            interleaved[i, 1] = 0
            if ids_e[0] != 0:
                queue_p = np.delete(queue_p, np.where(ids_p == ids_e[0])[0])
                ids_p = np.delete(ids_p, np.where(ids_p == ids_e[0])[0])
                interleaved[i, 2] = 1
            queue_e = np.delete(queue_e, 0)
            ids_e = np.delete(ids_e, 0)
        if order[i] == 1 and len(queue_p) > 0:
            interleaved[i, 0] = queue_p[0]
            interleaved[i, 1] = 1
            if ids_p[0] != 0:
                queue_e = np.delete(queue_e, np.where(ids_e == ids_p[0])[0])
                ids_e = np.delete(ids_e, np.where(ids_e == ids_p[0])[0])
                interleaved[i, 2] = 1
            queue_p = np.delete(queue_p, 0)
            ids_p = np.delete(ids_p, 0)
    return interleaved
  
        
  

### Step 4: Simulate User Clicks


In [0]:
# Generating the dataset of queries and clicks

queries = []
clicks = []
query = 'Q'
click = 'C'

with open('YandexRelPredChallenge.txt', 'r') as csvfile:
  reader = csv.reader(csvfile, delimiter='\t')
  for row in reader:
    if row[2] == query:
      queries.append(row)
    elif row[2] == click:
      clicks.append(row)

print("There are %i queries." % len(queries))
print("The shortest query contains %i documents." % (len(min(queries, key = len)) - 5))
print("The longest query contains %i documents." % (len(max(queries, key = len)) - 5))
print("There are %i clicks." % len(clicks))

# Optimizing the gamma and alpha parameters based on the Yandex query log 
# Using EM algorithm and formulas given by Markov  

def obtain_click_parameters(clicks,queries):
  #First, generate all the unique document-query pairs. Those are needed for keeping track of the alphas.
  uniquepairs = []
  for quer in queries:
    urlids = quer[5:11]
    for i in range(len(urlids)):
      unique = quer[3] + ',' + urlids[i]
      if unique not in [i[0] for i in uniquepairs]:
        uniquepairs.append([unique,1])
        count = count + 1
      else:
        indexx = uniquepairs.index([ind for ind in uniquepairs if unique in ind][0])
        uniquepairs[indexx][1] = uniquepairs[indexx][1] + 1
  #with open('uniquepairs.data', 'rb') as filehandle:  
  #    uniquepairs = pickle.load(filehandle)
  #Now, record all the clicked documents so that these checks don't have to be made inside the main loop.
  clicked = []
  clickid = []
  temp = 0
  tempclic = [0]
  count = 1
  for quer in queries:
    if quer[0] not in clickid:
      for clic in clicks:
        if clic[0] == quer[0]:
          if temp == int(clic[0]):
            tempclic.append(clic[3])
          else: 
            clicked.append(tempclic)
            temp = temp + 1
            tempclic = [temp]
            clickid.append(clic[0])
  clicked[0] = clicked[0][0:4]
  clicked.append([13868,'411'])
  clicked.append([13869])
  #Initialize the gammas and the zeros
  oldgammas = np.zeros(6)
  oldalphas = []
  gammas = np.zeros(6)
  alphas = []
  for i in range(len(uniquepairs)):
    temp = uniquepairs[i][0:2]
    temp.append(0)
    alphas.append(temp)
    oldalphas.append(temp)
  alphas = np.asarray(alphas)
  oldalphas = np.asarray(oldalphas)
  #Iterate the EM algorithm until convergence.
  goforward = True
  difference = 0.01
  iter = 0
  while (goforward):
    count = 0
    for quer in queries:
      id = quer[0]
      urlid = quer[5:11]
      qurlid = ''
      j = 0
      for url in urlid:
        qurlid = quer[3] + ',' + url
        alphaindex = np.where(alphas[:,0] == qurlid)
        alphaindex = alphaindex[0]
        if url in clicked[int(id)]:
          gammas[j] = gammas[j] + 1
          alphas[alphaindex,2] = float(alphas[alphaindex,2]) + 1
        else:
          gammas[j] = gammas[j] + oldgammas[j]*(1-float(oldalphas[alphaindex,2])) / (1 - oldgammas[j]*float(oldalphas[alphaindex,2]))  
          alphas[alphaindex,2] = float(alphas[alphaindex,2]) + float(oldalphas[alphaindex,2])*(1-oldgammas[j]) / (1 - oldgammas[j]*float(oldalphas[alphaindex,2]))  
        j = j + 1
      count = count + 1
      if (count % 5000) == 0:
          print(count)
    for j in range(len(alphas)):
      alphas[j][2] = (1 + float(alphas[j][2])) / (int(alphas[j][1])+2)
      oldalphas[j][2] = float(alphas[j][2])
    for j in range(len(gammas)):
      gammas[j] = (1+gammas[j])/(len(queries)+2)
      if (abs(gammas[j] - oldgammas[j]) < difference):
        goforward = False
      oldgammas[j] = gammas[j]
    iter = iter + 1
    print('iteration: %i' % iter)
    print(gammas)
  return gammas, alphas
  
#gammas, alphas = obtain_click_parameters(clicks,queries)

# after 10 iterations, these gammas were obtained
gammas = [0.50022533, 0.29569979, 0.22755148, 0.18252781, 0.14754686, 0.1253613]

# the learned alphas were discarded and replaced with the following value
alpha = 0.8

There are 42652 queries.
The shortest query contains 10 documents.
The longest query contains 10 documents.
There are 57348 clicks.


In [0]:
#Method that simulates clicking on a document having read it
def tosscoin(relev, alpha, ind):
  cointwo = random.random()
  if cointwo < (relev*alpha + (1-relev)*(1-alpha)):
    return ind, True
  else:
    return -1, False

#Method that simulates a position-based click model
def sim_click(interleaved,gammas,alpha):
  clickInd = -1    
  while (clickInd==-1):
    coin = random.random()
    for i in range(len(interleaved)):
      if coin < gammas[i]:
        clickInd, stop = tosscoin(interleaved[i,0],alpha,i)
        if stop:
          return clickInd
  return clickInd

#Method that simulates a random click model    
def sim_random_click(n):
  return random.randint(0,(n-1))

#Method that repeats the click simulation n times for a given interleaved list
#Runs it for either PBM or RND model, depending on boolean 'random'
def sim_click_model(interleaved, gammas, alpha, n, random = False):
  wins = np.zeros(n)
  for i in range(n):
    if (not random):
      indd = sim_click(interleaved,gammas,alpha)
      wins[i] = int(interleaved[indd,1])
    else: 
      indd = sim_random_click(len(interleaved))
      wins[i] = int(interleaved[indd,1])
  win = sum(wins) / n
  if win > 0.5:
    return 1
  else:
    return 0

### Step 5: Simulate Interleaving Experiment


#### Step 5a: Team-Draft Interleaving

In [0]:
def team_draft_interleaving(pairs, p_overlapping_rel, p_overlapping_not_rel, overlapping_samples):
  """ 
  Function performs all iterations of team draft interleaving.    
  Args:
    pairs: output of the function simulate_pairs
    p_overlapping_rel
    p_overlapping_not_rel
    overlapping_samples: number of times we sample potential overlaps in documents for each sample
  Returns:
    proportions: data structure that notes (delta_ERR, p1_positional, p1_random) for each sampled pair x overlapping_samples)           
  """
  proportions = []
  orders = algorithm_orders()
  n = 100  # the number of simulations
  for i in range(len(pairs)): # we iterate over every possible pair
    pair_i = np.asarray(pairs[i])
    delta_ERR = err(pair_i[0,:], theta) - err(pair_i[1,:], theta) # calculate delta_err
    if delta_ERR >= 0.05 and delta_ERR <= 0.95:
      for k in range(overlapping_samples): # if delta_err falls into desired bracket, we sample different combinations of overlapping in documents
        ids_i, overlappings_i = gen_ids(pair_i, p_overlapping_rel, p_overlapping_not_rel)
        result = np.zeros((3))
        PBM_wins = []
        RND_wins = [] 
        for j in range(len(orders)): # we iterate over every possible team draft order
          interleaving_ = interleaving(pair_i, ids_i, np.asarray(orders[j]), overlappings_i) # we calculate interleaving for every combination of pair, order and overlapping
          PBM_win = sim_click_model(interleaving_, gammas, alpha, n)  # we simulate clicks for positional click model
          RND_win = sim_click_model(interleaving_, gammas, alpha, n, random=True) # and random click model
          PBM_wins.append('E' if PBM_win == 0 else 'P')
          RND_wins.append('E' if RND_win == 0 else 'P')
        result[0] = delta_ERR
        result[1] = PBM_wins.count('E') / len(PBM_wins)
        result[2] = RND_wins.count('E') / len(RND_wins)
        proportions.append(result)
  return np.asarray(proportions)

**Step 5b: Probabilistic Interleaving**

In [0]:
def probabilistic_interleaving(pairs, p_overlapping_rel, p_overlapping_not_rel, overlapping_samples):
    """ Function performs all iterations of probabilistic interleaving.

        Args:
            pairs: output of the function simulate_pairs
            p_overlapping_rel
            p_overlapping_not_rel
            overlapping_samples: number of times we sample potential overlaps in documents for each sample
        Returns:
            proportions: data structure that notes (delta_ERR, p1_positional, p1_random) for each sampled pair x overlapping_samples)

    """
    proportions = []
    orders = algorithm_orders()
    n = 100  # the number of simulations
    for i in range(len(pairs)): # we iterate over every possible pair
        pair_i = np.asarray(pairs[i])
        delta_ERR_ = err(pair_i[0,:], theta) - err(pair_i[1,:], theta) # calculate delta_err
        if delta_ERR_ >= 0.05 and delta_ERR_ <= 0.95:
            for m in range(overlapping_samples): # if delta_err falls into desired bracket, we sample different combinations of overlapping in documents
                ids_i, overlappings_i = gen_ids(pairs[i], p_overlapping_rel, p_overlapping_not_rel)
                result = np.zeros((3))
                PBM_wins = []
                RND_wins = []
                total = 0
                prob_pairs, prob_ids, prob_of_ranking = probabilistic_document_order(pair_i, ids_i)
                for j in range(len(orders)): # we iterate over every possible team draft order
                    for k in range(len(prob_pairs)): # we iterate over every order in which algorithms can send their documents
                        interleaving_ = interleaving(pair_i, ids_i, np.asarray(orders[j]), overlappings_i)
                        PBM_win = sim_click_model(interleaving_, gammas, alpha, n) # we simulate clicks
                        RND_win = sim_click_model(interleaving_, gammas, alpha, n, random=True)
                        PBM_wins.append(prob_of_ranking[k] if PBM_win == 0 else 0) # since we analyse 1 of every combination of probabilstic interleaving, we weight win by probability of that interleaving occuring
                        RND_wins.append(prob_of_ranking[k] if RND_win == 0 else 0)
                        total += prob_of_ranking[k]
                result[0] = delta_ERR_
                result[1] = sum(PBM_wins) / total
                result[2] = sum(RND_wins) / total
                proportions.append(result)
    return np.asarray(proportions)

In [0]:
team_draft_results = team_draft_interleaving(pairs, p_overlapping_rel, p_overlapping_not_rel, overlapping_samples)
probabilistic_results = probabilistic_interleaving(pairs, p_overlapping_rel, p_overlapping_not_rel, overlapping_samples)

### Step 6: Compute Sample Size

In [0]:
def one_sided_proportion_test(p1, p0=0.5, alpha=0.05, beta=0.10):
    """Calculate the sample size using a proportion test for the 1-sided case.

    Args:
        p1: the proportion of times E wins over P
        p0: baseline proportion to compare p1 with
        alpha: probability of false positive
        beta: probability of false negative
    Returns:
        the sample size required
    """
    if p1 - p0 < 0.0000001:
        return np.inf
    
    z_alpha = norm.ppf(1.0 - alpha)
    z_beta = norm.ppf(1.0 - beta)
    
    n = np.int_(np.ceil(p0 * (1 - p0) * ((z_alpha + z_beta * np.sqrt((p1 * (1 - p1)) / (p0 * (1 - p0)))) / (p1 - p0)) ** 2))

    return n

#### Step 6a: Team-Draft Interleaving

In [0]:
# get the proportions for the group
team_draft_delta_ERRs = team_draft_results[:, 0]
team_draft_PBM_proportions = team_draft_results[:, 1]
team_draft_RND_proportions = team_draft_results[:, 2]

team_draft_PBM_sample_sizes = []
team_draft_RND_sample_sizes = []

for i in range(len(team_draft_PBM_proportions)):
    team_draft_PBM_sample_sizes.append(one_sided_proportion_test(team_draft_PBM_proportions[i]))
    team_draft_RND_sample_sizes.append(one_sided_proportion_test(team_draft_RND_proportions[i]))

#### Step 6b: Probabilistic Interleaving

In [0]:
# get the proportions for the group
probabilistic_delta_ERRs = probabilistic_results[:, 0]
probabilistic_PBM_proportions = probabilistic_results[:, 1]
probabilistic_RND_proportions = probabilistic_results[:, 2]

probabilistic_PBM_sample_sizes = []
probabilistic_RND_sample_sizes = []

for i in range(len(probabilistic_PBM_proportions)):
    probabilistic_PBM_sample_sizes.append(one_sided_proportion_test(probabilistic_PBM_proportions[i]))
    probabilistic_RND_sample_sizes.append(one_sided_proportion_test(probabilistic_RND_proportions[i]))

#### Step 6c: Group the results

In [0]:
team_draft_PBM_sample_sizes_grouped = [[] for treshold in thresholds]
team_draft_RND_sample_sizes_grouped = [[] for treshold in thresholds]

for i in range(len(team_draft_PBM_sample_sizes)):
    group = [j for j in range(len(thresholds)) if (team_draft_delta_ERRs[i] >= thresholds[j][0] and team_draft_delta_ERRs[i] < thresholds[j][1])]
    if len(group) > 0:
        team_draft_PBM_sample_sizes_grouped[group[0]].append(team_draft_PBM_sample_sizes[i])
        team_draft_RND_sample_sizes_grouped[group[0]].append(team_draft_RND_sample_sizes[i])
        
probabilistic_PBM_sample_sizes_grouped = [[] for treshold in thresholds]
probabilistic_RND_sample_sizes_grouped = [[] for treshold in thresholds]

for i in range(len(probabilistic_PBM_sample_sizes)):
    group = [j for j in range(len(thresholds)) if (probabilistic_delta_ERRs[i] >= thresholds[j][0] and probabilistic_delta_ERRs[i] < thresholds[j][1])]
    if len(group) > 0:
        probabilistic_PBM_sample_sizes_grouped[group[0]].append(probabilistic_PBM_sample_sizes[i])
        probabilistic_RND_sample_sizes_grouped[group[0]].append(probabilistic_RND_sample_sizes[i])

### Step 7: Analysis

#### Step 7a: Team-Draft Interleaving

In [0]:
print('Position Based Model, Team-Draft interleaving')
print('ΔERR                 Min       Median    Max       ')
for i in range(len(thresholds)): # loop for each group/bin
    if len(team_draft_PBM_sample_sizes_grouped[i]) > 0:
        min_ = min(team_draft_PBM_sample_sizes_grouped[i])
        median_ = np.int_(np.median(team_draft_PBM_sample_sizes_grouped[i]))
        if median_ < 0:
            median_ = np.inf
        max_ = max(team_draft_PBM_sample_sizes_grouped[i])
    
        print(f'[{thresholds[i][0]:.2f}, {thresholds[i][1]:.2f}) (n={len(team_draft_RND_sample_sizes_grouped[i]): <2}): {min_: <9} {median_: <9} {max_: <9}')

print()
print('Random Click Model, Team-Draft interleaving')
print('ΔERR                 Min       Median    Max       ')
for i in range(len(thresholds)): # loop for each group/bin
    if len(team_draft_RND_sample_sizes_grouped[i]) > 0:
        min_ = min(team_draft_RND_sample_sizes_grouped[i])
        median_ = np.int_(np.median(team_draft_RND_sample_sizes_grouped[i]))
        if median_ < 0:
            median_ = np.inf
        max_ = max(team_draft_RND_sample_sizes_grouped[i])
    
        print(f'[{thresholds[i][0]:.2f}, {thresholds[i][1]:.2f}) (n={len(team_draft_RND_sample_sizes_grouped[i]): <2}): {min_: <9} {median_: <9} {max_: <9}')

Position Based Model, Team-Draft interleaving
ΔERR                 Min       Median    Max       
[0.05, 0.10) (n=8 ): 92        inf       inf      
[0.10, 0.20) (n=8 ): 50        92        211      
[0.20, 0.30) (n=8 ): 20        50        92       
[0.30, 0.40) (n=28): 20        20        31       
[0.40, 0.50) (n=16): 10        20        20       
[0.50, 0.60) (n=4 ): 20        20        20       
[0.60, 0.70) (n=16): 10        14        20       

Random Click Model, Team-Draft interleaving
ΔERR                 Min       Median    Max       
[0.05, 0.10) (n=8 ): 20        inf       inf      
[0.10, 0.20) (n=8 ): 31        532       inf      
[0.20, 0.30) (n=8 ): 92        532       inf      
[0.30, 0.40) (n=28): 6         92        inf      
[0.40, 0.50) (n=16): 14        211       inf      
[0.50, 0.60) (n=4 ): 6         inf       inf      
[0.60, 0.70) (n=16): 31        532       inf      


#### Step 7b: Probabilistic Interleaving

In [0]:
print('Position Based Model, Probabilistic interleaving')
print('ΔERR                 Min       Median    Max       ')
for i in range(len(thresholds)): # loop for each group/bin
    if len(probabilistic_PBM_sample_sizes_grouped[i]) > 0:
        min_ = min(probabilistic_PBM_sample_sizes_grouped[i])
        median_ = np.int_(np.median(probabilistic_PBM_sample_sizes_grouped[i]))
        if median_ < 0:
            median_ = np.inf
        max_ = max(probabilistic_PBM_sample_sizes_grouped[i])
    
        print(f'[{thresholds[i][0]:.2f}, {thresholds[i][1]:.2f}) (n={len(probabilistic_PBM_sample_sizes_grouped[i]): <2}): {min_: <9} {median_: <9} {max_: <9}')

print()
print('Random Click Model, Probabilistic interleaving')
print('ΔERR                 Min       Median    Max       ')
for i in range(len(thresholds)): # loop for each group/bin
    if len(probabilistic_RND_sample_sizes_grouped[i]) > 0:
        min_ = min(probabilistic_RND_sample_sizes_grouped[i])
        median_ = np.int_(np.median(probabilistic_RND_sample_sizes_grouped[i]))
        if median_ < 0:
            median_ = np.inf
        max_ = max(probabilistic_RND_sample_sizes_grouped[i])
    
        print(f'[{thresholds[i][0]:.2f}, {thresholds[i][1]:.2f}) (n={len(probabilistic_RND_sample_sizes_grouped[i]): <2}): {min_: <9} {median_: <9} {max_: <9}')

Position Based Model, Probabilistic interleaving
ΔERR                 Min       Median    Max       
[0.05, 0.10) (n=8 ): 86        inf       inf      
[0.10, 0.20) (n=8 ): 54        99        186      
[0.20, 0.30) (n=8 ): 32        44        59       
[0.30, 0.40) (n=28): 20        20        23       
[0.40, 0.50) (n=16): 13        17        21       
[0.50, 0.60) (n=4 ): 13        15        17       
[0.60, 0.70) (n=16): 12        15        17       

Random Click Model, Probabilistic interleaving
ΔERR                 Min       Median    Max       
[0.05, 0.10) (n=8 ): 15        5389      inf      
[0.10, 0.20) (n=8 ): 53        1095      inf      
[0.20, 0.30) (n=8 ): 199       839       2330743  
[0.30, 0.40) (n=28): 15        700       inf      
[0.40, 0.50) (n=16): 7         604       inf      
[0.50, 0.60) (n=4 ): 7         153       112717   
[0.60, 0.70) (n=16): 156       886       inf      


#### Step 7c: Results and Conclusions

##### Correlation between ERR (offline metric) and proportions of wins (online metric)

In [0]:
slope, intercept, r_value, p_value, std_err = stats.linregress(team_draft_delta_ERRs, team_draft_PBM_proportions)
print(f"For Team-Draft interleaving, with the PBM click model, there is a {'significant' if p_value < 0.05 else 'not significant'}, {'positive' if slope > 0 else 'negative'} relationship between ΔERR and the proportion of wins of E over P with coefficient={slope:.4f} and p = {p_value}.")
slope, intercept, r_value, p_value, std_err = stats.linregress(team_draft_delta_ERRs, team_draft_RND_proportions)
print(f"For Team-Draft interleaving, with the RND click model, there is a {'significant' if p_value < 0.05 else 'not significant'}, {'positive' if slope > 0 else 'negative'} relationship between ΔERR and the proportion of wins of E over P with coefficient={slope:.4f} and p = {p_value}.")

print()

slope, intercept, r_value, p_value, std_err = stats.linregress(probabilistic_delta_ERRs, probabilistic_PBM_proportions)
print(f"For Probabilistic interleaving, with the PBM click model, there is a {'significant' if p_value < 0.05 else 'not significant'}, {'positive' if slope > 0 else 'negative'} relationship between ΔERR and the proportion of wins of E over P with coefficient={slope:.4f} and p = {p_value}.")
slope, intercept, r_value, p_value, std_err = stats.linregress(probabilistic_delta_ERRs, probabilistic_RND_proportions)
print(f"For Probabilistic interleaving, with the RND click model, there is a {'significant' if p_value < 0.05 else 'not significant'}, {'positive' if slope > 0 else 'negative'} relationship between ΔERR and the proportion of wins of E over P with coefficient={slope:.4f} and p = {p_value}.")

For Team-Draft interleaving, with the PBM click model, there is a significant, positive relationship between ΔERR and the proportion of wins of E over P with coefficient=0.4629 and p = 5.2169188679920105e-24.
For Team-Draft interleaving, with the RND click model, there is a not significant, positive relationship between ΔERR and the proportion of wins of E over P with coefficient=0.0175 and p = 0.8523434258164561.

For Probabilistic interleaving, with the PBM click model, there is a significant, positive relationship between ΔERR and the proportion of wins of E over P with coefficient=0.4547 and p = 2.0313011658449783e-27.
For Probabilistic interleaving, with the RND click model, there is a not significant, positive relationship between ΔERR and the proportion of wins of E over P with coefficient=0.0112 and p = 0.8737685640587385.


When looking at the relationship between the *ΔERR* (our offline measure) and the proportion of wins of algorithm E over P *p1* (our online measure), we observe a significant (at the 95% confidence interval), positive relationship for the PBM click model (and a non-significant, negative relationship for the Random click model).

This would suggest that our offline measure does indeed inform our online measure!

##### (Non-)Significant differences between the PBM click model and the Random click model

In [0]:
# discard all entries where the sample size is infinite
PBM_sample_sizes = team_draft_PBM_sample_sizes + probabilistic_PBM_sample_sizes
RND_sample_sizes = team_draft_RND_sample_sizes + probabilistic_RND_sample_sizes

new_PBM_sample_sizes = []
new_RND_sample_sizes = []

for i in range(len(PBM_sample_sizes)):
   if PBM_sample_sizes[i] != np.inf and RND_sample_sizes[i] != np.inf:
        new_PBM_sample_sizes.append(PBM_sample_sizes[i])
        new_RND_sample_sizes.append(RND_sample_sizes[i])

# compute the t-test
statistic, p_value = ttest_ind(new_PBM_sample_sizes, new_RND_sample_sizes, equal_var=False)

print(f"The difference in sample sizes between the PBM click modeland the\nRandom click model is {'significant' if p_value / 2 < 0.05 else 'not significant'} (1-sided t-test, p: {p_value / 2:.4f})")

The difference in sample sizes between the PBM click modeland the
Random click model is significant (1-sided t-test, p: 0.0374)


In [0]:
PBM_proportions = team_draft_PBM_proportions + probabilistic_PBM_proportions
RND_proportions = team_draft_RND_proportions + probabilistic_RND_proportions

# compute the t-test for the proportion of wins
statistic, p_value = ttest_ind(PBM_proportions, RND_proportions, equal_var=False)

print(f"The difference in proportion of wins p1 between the PBM click model and\nthe Random click model is {'significant' if p_value / 2 < 0.05 else 'not significant'} (1-sided t-test, p: {p_value / 2})")

The difference in proportion of wins p1 between the PBM click model and
the Random click model is significant (1-sided t-test, p: 7.897870634839043e-24)


When looking at the results of our interleaving experiments, one immediately notices the differences between the PBM click model and the Random click model, for both Team-Draft and Probabilistic interleaving, in terms of the computed sample sizes.

As expected, with the Random click model, many more impressions (sometimes infinitely many if p1 ~ p0) are needed for our interleaving experiments, when compared to the PBM click model. This difference is significant (at the 95% confidence interval).

This is because, essentially, the Random click model ignores any differences in rank (and thus differences in relevance) between the documents presented in the interleaving. As such, the values for p1 will be closer to (or even below) 0.5, when compared to the PBM click model, resulting in larger sample sizes required.

##### (Non-)Significant differences between Team-Draft interleaving and Probabilistic interleaving

In [0]:
# discard all entries where the sample size is infinite
new_team_draft_PBM_sample_sizes = []
new_probabilistic_PBM_sample_sizes = []

for i in range(len(team_draft_PBM_sample_sizes)):
   if team_draft_PBM_sample_sizes[i] != np.inf and probabilistic_PBM_sample_sizes != np.inf:
        new_team_draft_PBM_sample_sizes.append(team_draft_PBM_sample_sizes[i])
        new_probabilistic_PBM_sample_sizes.append(probabilistic_PBM_sample_sizes[i])

# compute the t-test for the sample sizes
statistic, p_value = ttest_ind(new_team_draft_PBM_sample_sizes, new_probabilistic_PBM_sample_sizes, equal_var=False)

print(f"The difference in sample sizes between Team-Draft interleaving and\nProbabilistic interleaving for the PBM click model\nis {'significant' if p_value / 2 < 0.05 else 'not significant'} (1-sided t-test, p: {p_value / 2:.4f})")

The difference in sample sizes between Team-Draft interleaving and
Probabilistic interleaving for the PBM click model
is not significant (1-sided t-test, p: 0.2068)


In [0]:
# compute the t-test for the proportion of wins
statistic, p_value = ttest_ind(team_draft_PBM_proportions, probabilistic_PBM_proportions, equal_var=False)

print(f"The difference in proportion of wins p1 between Team-Draft\ninterleaving and Probabilistic interleaving for the PBM\nclick model is {'significant' if p_value / 2 < 0.05 else 'not significant'} (1-sided t-test, p: {p_value / 2:.4f})")

The difference in proportion of wins p1 between Team-Draft
interleaving and Probabilistic interleaving for the PBM
click model is not significant (1-sided t-test, p: 0.3260)


Moreover, while Probabilistic interleaving does result in lower sample sizes for most bins, the differences are rather small and non-significant (at the 95% confidence interval). They definitely don't differ by a factor of 2^3=8 (where Probabilistic would need 8 times fewer samples than Team-Draft) as put forward in our design proposal.


#### Step 7d: Limitations and Possible Improvements

Despite the results that confirmed some of the expectations we have set out in the theoretical paper, there are still some limitations and room for possible improvement. 

Most importantly, there are several restrictions that were imposed by the design of the experiment. Due to the coarse relevance grading scale and low cut-off rank, there were fewer combinations of ranking pairs and, hence, fewer possible ΔERR results and possible intereleaved lists. This led to a small number of samples per ΔERR bin, and improving this aspect might make the findings of this experiment more stable. Also as a result of this limitation, we observed a weaker difference between team-draft and probabilistic interleaving methods. 

There is also room for improvement in terms of broading the diversity of metrics and algorithms employed in this experiment. Firstly, the number of metrics to evaluate a ranking could be enlarged: next to Expected Reciprocal Rank, Discounted Cumulative Gain, Rank-Based Precision or Mean Average Precision could be included. Secondly, more interleaving methods could be evaluated: Balanced, which is more primitive than Team-Draft and Probabilistic methods, and Optimized, which is reported to improve on the Probabilistic approach. 

Perhaps the most important improvement needed for this experiment is the inclusion of more sophisticated click models that accurately mimic user behavior. Since in our experiment, attractiveness of the document was substitued by the crude relevance score, other algorithms might capture these differences. This might prove to be challenging because it would require running the experiment on more detailed datasets rather than artificially simulated rankings produced for a single query, as is the case in our paper. 

Nevertheless, while inclusion of the above mentioned improvements might change the outcomes of the experiment, we believe that we confirmed our initial hypothesis of negative correlation between delta_ERR / predicted sample size and presented a case for predicting sample sizes needed in online experiments. 

#### References

Azarbonyad, H., & Kanoulas, E. (2016, September). Power Analysis for Interleaving Experiments by means of Offline Evaluation. In Proceedings of the 2016 ACM International Conference on the Theory of Information Retrieval (pp. 87-90). ACM.

Chapelle, O., Metlzer, D., Zhang, Y., & Grinspan, P. (2009, November). Expected reciprocal rank for graded relevance. In Proceedings of the 18th ACM conference on Information and knowledge management (pp. 621-630). ACM.

Chuklin, A., Markov, I., & Rijke, M. D. (2015). Click models for web search. Synthesis Lectures on Information Concepts, Retrieval, and Services, 7(3), 1-115.

Dupret, G. (2011, October). Discounted cumulative gain and user decision models. In International Symposium on String Processing and Information Retrieval (pp. 2-13). Springer, Berlin, Heidelberg.

Hofmann, K., Whiteson, S., & De Rijke, M. (2011, October). A probabilistic method for inferring preferences from clicks. In Proceedings of the 20th ACM international conference on Information and knowledge management (pp. 249-258). ACM.

Radlinski, F., & Craswell, N. (2013, February). Optimized interleaving for online retrieval evaluation. In Proceedings of the sixth ACM international conference on Web search and data mining (pp. 245-254). ACM.